In [ ]:
'''------------Import Required Modules------------'''
import tensorflow as tf     
import keras                
from keras import layers 

In [ ]:
'''------------Separate our Data------------'''
# Why would we want to separate data?
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
'''------------Normalize Data------------'''
# What happens if we don't normalize? 
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
'''------------Create a Sequential Model------------'''
model = keras.Sequential(
    [
        layers.Flatten(input_shape=(28,28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ]
)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

'''------------What are the Dimensions of the Network?------------'''
model.summary()

In [ ]:
'''------------Train the Neural Network------------'''
history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=32
)

In [ ]:
'''------------How Did the Model Perform?------------'''
predictions = model.predict(x_test[:30])
print(predictions.argmax(axis=1))
print(y_test[:30])